In [1]:
# Load the environment variables
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
# Configuration for the Worker agent

from agents import ModelSettings
from datetime import datetime

agent_name = "Basic Worker"

agent_instructions = """
You are a single-step Worker. Execute the given task directly and return the best possible result.
Do not ask clarifying questions. Do not handoff, delegate, or call tools unless explicitly provided.
Prefer concise, correct outputs. Use Markdown formatting when helpful.
"""

agent_model = "gpt-4o-mini"

agent_model_settings=ModelSettings(
    model=agent_model,
    temperature=0.2,
    max_tokens=1000
)

agent_session_name  = agent_name + datetime.now().strftime("%Y-%m-%dT%H:%M:%SU%s")

In [3]:
# Define the Worker agent

from agents import Agent, Runner, trace

worker = Agent(
    name=agent_name,
    instructions=agent_instructions,
    model_settings=agent_model_settings
)

In [4]:
# Give the agent the ability to remember from one interaction to the next.

from agents import SQLiteSession

agent_session = SQLiteSession(agent_session_name)

In [5]:
async def assign_task(task: str, enable_memory: bool) -> str:
    """
    Asssign a task to the agent and receive its response in return.
    """

    # Check that Worker agent exists
    if worker is None:
        return "Agent has not been created yet."
        
    try:
        result = None
        with trace(worker.name):
            if enable_memory:
                result = await Runner.run(worker, task, session=agent_session)
            else:
                result = await Runner.run(worker, task)
        return result.final_output
    except Exception as e:
        return f"Error: {e}"


In [6]:
from agents import Agent, ModelSettings, Runner, SQLiteSession, trace
from datetime import datetime


class WorkerPattern:

    def __init__(self, agent_name: str = None, agent_instructions: str = None, 
                 agent_model: str = "gpt-4o-mini", 
                 agent_temperature: float = 0.2, 
                 agent_max_tokens: int = 1000,
                 agent_has_memory: bool = False) -> None:
        """
        Initialize the object and set the private parameters that govern the worker agent.
        """

        # Generate a timestamp string for unique naming
        now_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%SU%s")

        # Assign the agent a name
        self.__agent_name = agent_name or f"WorkerAgent_{now_string}"
        
        # Generate a unique memory Session name
        self.__agent_session_name = f"{self.__agent_name}__SESSION_{now_string}"

        # Set the agent's instructions prompt
        self.__agent_instructions = agent_instructions or """
            You are a single-step Worker. Execute the given task directly and return the best possible result.
            Do not ask clarifying questions. Do not handoff, delegate, or call tools unless explicitly provided.
            Prefer concise, correct outputs. Use Markdown formatting when helpful.
        """

        # Set additional agent parameters
        self.__agent_model = agent_model
        self.__agent_temperature = float(agent_temperature)
        self.__agent_max_tokens = int(agent_max_tokens)
        self.__agent_has_memory = bool(agent_has_memory)

        self.__worker_agent = None
        self.__agent_session = None


    def __create_agent(self):
        """
        Create the worker agent and a session (if the agent is meant to have memory).
        """

        agent_model_settings = ModelSettings(
            temperature=self.__agent_temperature,
            max_tokens=self.__agent_max_tokens
        )

        self.__worker_agent = Agent(
            name=self.__agent_name,
            instructions=self.__agent_instructions,
            model=self.__agent_model,
            model_settings=agent_model_settings
        )

        if self.__agent_has_memory:
            self.__agent_session = SQLiteSession(self.__agent_session_name)
        

        
    async def assign_task(self, task: str, enable_memory: bool) -> str:
        """
        Asssign a task to the agent and receive its response in return.
        """

        # Check that Worker agent exists
        if self.__worker_agent is None:
            self.__create_agent()
            
        try:
            result = None
            with trace(self.__worker_agent.name):
                if enable_memory:
                    result = await Runner.run(self.__worker_agent, task, session=self.__agent_session)
                else:
                    result = await Runner.run(self.__worker_agent, task, session=None)
            return result.final_output
        except Exception as e:
            return f"Error: {e}"
    

In [ ]:
# Create a gradio interface to interact with the worker agent

import gradio as gr

# Set this to True if you want to test WorkerPattern object, False to test the standalone methods.
use_worker_pattern = False

with gr.Blocks(title="Worker — Agentic Design Patterns") as demo:

    wp = WorkerPattern(agent_has_memory=True) if use_worker_pattern else None

    gr.Markdown("""
    # 🛠️ Worker Pattern
    
    This is a simple worker agent that can be used to perform single-step execution tasks without delegation or validation. Ask, and it will do its best to complete the task.
    """)

    with gr.Column():
        with gr.Row():
            task = gr.Textbox(label="Task", placeholder="What do you want the Worker to do?", lines=4, scale=9)
            with gr.Column(scale=1):
                enable_memory = gr.Checkbox(label="Remember Stuff")
                run_btn = gr.Button("Ask", variant="primary", scale=1)
        out = gr.Markdown(label="Result")

    async def on_run(task:str, memory: bool):
        run_btn.interactive = False  # disable button while running
        try:
            if wp:
                return await wp.assign_task(task, memory)
            else:
                return await assign_task(task, memory)
        finally:
            run_btn.interactive = True  # re-enable button

    run_btn.click(on_run, inputs=[task, enable_memory], outputs=[out])
    task.submit(on_run, inputs=[task, enable_memory], outputs=[out])

demo.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
